# Storage

## Introduction
>Pods and containers are __ephemeral__, i.e. they are short-lived.

### Merits of pods and containers
- Easy to create and destroy
- Immutable and easy to reason about.
- Easy to parallelise.

### Demerits of pods and containers
- When a pod is removed, all of the data created by it on the local disk are also removed.
- Each container is a separate entity. Therefore, data sharing between containers is not possible, unless `Volume`s are employed.

## Volumes

`Volume`s in `k8s` are slightly different from those in `Docker`.

### Brief overview of `volume`s in `Docker`
- The directory is located on the `disk` or in another `container`.
- `Volume`s are mounted in containers during runtime.
- Data sharing across instances (or using `cloud` storage) is possible via `drivers` (for more information, see [here](https://docs.docker.com/storage/volumes/#share-data-among-machines)).

Although the offerings resolve some of our problems, the feature set is quite limited and inadequate for large-scale deployments.

### High-level features
- __`Volume`s can be mounted simultaneously.__
- __`Ephemeral volume`s have lifetimes similar to those of their `POD`s.
- __`Persistent volume`s have lifetimes independent of those of their `POD`s.
- __Data is available across `container`s restart,__ handled by `kubelet`.

Volumes can be attached to pods via the following:
- `.spec.volumes`: specifies which `volumes` to use.
- `.spec.containers[*].volumeMounts`: specifies  where and which volume to mount in a specific container.

Consider the below example using a `DaemonSet`:

```
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
  labels:
    k8s-app: fluentd-logging
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
        # Here we can mount them with `name` matching
        # Ephemeral Volumes
        volumeMounts:
        - name: varlog
          mountPath: /var/log
        - name: varlibdockercontainers
          mountPath: /var/lib/docker/containers
          readOnly: true
      terminationGracePeriodSeconds: 30
      # Here we define our volumes
      # Data from POD will be mounted
      volumes:
      - name: varlog
        hostPath:
          path: /var/log
      - name: varlibdockercontainers
        hostPath:
          path: /var/lib/docker/container

### Example

- Create a .yaml file for creating the DeamonSet.
- Create it, and use the describe command from kubectl to observe the volumes.
- Go to the minikube dashboard, and search for the volumes.

In [1]:
# Create the DaemonSet
!kubectl

daemonset.apps/fluentd-elasticsearch created


In [4]:
# Use the describe command
!kubectl

Name:           fluentd-elasticsearch
Selector:       name=fluentd-elasticsearch
Node-Selector:  <none>
Labels:         k8s-app=fluentd-logging
Annotations:    deprecated.daemonset.template.generation: 1
Desired Number of Nodes Scheduled: 3
Current Number of Nodes Scheduled: 3
Number of Nodes Scheduled with Up-to-date Pods: 3
Number of Nodes Scheduled with Available Pods: 3
Number of Nodes Misscheduled: 0
Pods Status:  3 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  name=fluentd-elasticsearch
  Containers:
   fluentd-elasticsearch:
    Image:      quay.io/fluentd_elasticsearch/fluentd:v2.5.2
    Port:       <none>
    Host Port:  <none>
    Limits:
      memory:  200Mi
    Requests:
      cpu:        100m
      memory:     200Mi
    Environment:  <none>
    Mounts:
      /var/lib/docker/containers from varlibdockercontainers (ro)
      /var/log from varlog (rw)
  Volumes:
   varlog:
    Type:          HostPath (bare host directory volume)
    Path:          /var

The dashboard should have an appearance similar to that in the figure below, with the volumes mounted in the container:
<p><img src=images/volume.png></p>

### Volume types

`kubernetes` provides a few integrations for standard volumes:
- `AWS Elastic Block Store`
- Microsoft's Azure `Disk` and `File`
- Self-hosted `cephfs`
- [Google Cloud Persistent Disk](https://kubernetes.io/docs/concepts/storage/volumes/#gcepersistentdisk)

An example config with `awsElasticBlockStore` is shown below:

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test-ebs
spec:
  containers:
  - image: k8s.gcr.io/test-webserver
    name: test-container
    volumeMounts:
    - mountPath: /test-ebs
      name: test-volume
  volumes:
  - name: test-volume
    # This AWS EBS volume must already exist.
    awsElasticBlockStore:
      volumeID: "<volume id>"
      fsType: ext4

For detailed information on volume types, check [here](https://kubernetes.io/docs/concepts/storage/volumes/#volume-types).

__Note that these are all ephemeral `volume`s; therefore, they will only live as long as the pod.__

### Persistent volumes

> `Kubernetes` provides two resources for managing persistent storage: `PersistentVolume` and `PersistentVolumeClaim`.

These allow us to abstract how storage is provided and expended.

#### PersistentVolume

A persistent volume refers to a storage volume in a cluster provisioned by an administrator. Persistent volumes are 'physical' volumes on the host machine.

*Characteristics of persistent volumes*

- They have resources in the cluster (similar to `Node`s).
- They are volume plugins similar to `volume`s described above.
- __Their lifetime is independent of that of any pod.__
- When bounded, they can be used similarly to `volume`s.

#### PersistentVolumeClaim

A persistent volume claim (PVC) is a request for the platform to create a PV on your behalf. To do this, it will search for any 'available' PersistentVolume that meets the specified requirements.

*Features*
- Conceptually similar to `POD`s:
    - `POD`s consume `Node` resources, whereas `PVC`s consume `PV` resources.
    - `POD`s can request specific resources (e.g. RAM), whereas `PVC`s can request specific sizes and access modes (e.g. `read` and `write`).
    
#### Provisioning

`PV`s can be provisioned in two ways:
- `statically`: cluster admin creates `PV`s for consumption.
- `dynamically`: cluster attempts to dynamically provision appropriate `PV`s based on the `PersistentVolumClaim`'s __`StorageClasses`__ (the administrator has to provision the `StorageClass`, which will be __described later__).

*Features*
- __`Dynamic provision` will always match the requirements of the `PVC`.__
- __`Static Provision` must at least match the given claim__ (e.g. a claim of `50GB` might be given `100Gb`).

#### Reclaim Policy

When a user is finished with a volume, they can delete the PVC objects from the API, which allows the reclamation of the resource.

There are three approaches for reclaiming the resource:
- [`retain`](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#retain): leave the data as-is (leave `PersistentVolume` and external storage.
- [`delete`](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#delete): delete `PersistentVolume` __and external storage__. This is the default approach for `Dynamic Provisioning`, although it is configurable.
- [`recycle`](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#recycle): (__now deprecated; dynamic provisioning should be used instead__).

Volumes can exist in any of the four states:

- `Available`: the volume is ready to be bound to a pod.
- `Bound`: the CLI shows the pod to which the `PersistentVolume` is bound.
- `Released`: the `PesistentVolumeClaim` has ended; however, the resource is yet to be reclaimed by the `cluster`.
- `Failed`: reclamation failed.

#### Specifying `PersistentVolume`

As with pods, PVs are created using `.yaml` config files, as shown below:

```
apiVersion: v1
kind: PersistentVolume
metadata:
  name: task-pv-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 10Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/mnt/data"
```

Below, we describe some common arguments that you can pass:
1. `.spec.capacity.storage`: request `10GB` of storage. Currently, only storage can be requested (see [`k8s` resource model for description of units](https://github.com/kubernetes/community/blob/master/contributors/design-proposals/scheduling/resources.md)).
2. `.spec.volumeMode`: either `FileSystem` (default) or `Block`.
    - `FileSystem`: directory mounted in pods.
    - `Block`: uses a raw block of storage (without a filesystem created). The `Block` option is rarely used as applications need to know how to access `raw` data (see [here](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#raw-block-volume-support) for an example).
3. `.spec.accessModes`: stipulates how data can be accessed.
    - `ReadWriteOnce`: volume mounted as `rw` by a single `Node`.
    - `ReadOnlyMany`: can be mounted by many `nodes`, but the data can only be read.
    - `ReadWriteMany`: same as above; however, `rw` access is granted (applications may need to handle possible data races).
    - `ReadWriteOncePod`: a single pod with `rw` access.
      
The modes above differ based on the type of `PersistentVolume` provider. A few of them are shown below:

![](./images/modes_providers.png)

4. __`.spec.storageclassName`__: specifies `StorageClass`. If left unspecified, __no `StorageClass` will be specified, and only a `PV` without one can be matched to the `POD`.__
5. `.spec.mountOptions`: __not supported by all types.__. It specifies how to mount the `disk` and can left as is.

#### Example

1. Create a .yaml file with the configuration above for creating a PersistentVolume resource.
2. `apply` it using the right `kubectl`.
3. Observe the status of the volume.

In [8]:
!kubectl

persistentvolume/task-pv-volume created


In [11]:
!kubectl

NAME             CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM   STORAGECLASS   REASON   AGE
task-pv-volume   10Gi       RWO            Retain           Available           manual                  13m


## StorageClasses

> A StorageClass provides a way for administrators to describe the 'classes' of storage they offer.

StorageClasses allow us to __dynamically provision storage__ and serve as templates for new `PersistentVolume`s.

As is conventional, these are defined using `.yaml` files and can be referred to by `PV` config files.

Consider the example below:

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: standard
provisioner: kubernetes.io/aws-ebs
parameters:
  type: gp2
reclaimPolicy: Retain
allowVolumeExpansion: true
mountOptions:
  - debug
volumeBindingMode: Immediate

> The `.metadata.name` value allows users to request this `StorageClass`.

#### Mandatory fields

> `provisioner`: which volume plugin is used for provisioning `PV`.

The most common ones are shipped with `k8s` under the `kubernetes.io` prefix. For example,

- `local`: `kubernetes.io/no-provisioner` (create `PV`s dynamically from the local resources).

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: local-storage
provisioner: kubernetes.io/no-provisioner
volumeBindingMode: WaitForFirstConsumer

- `GCEPersistentDisk`: Persistent disk from Google Cloud.

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: slow
provisioner: kubernetes.io/gce-pd
parameters:
  type: pd-standard
  fstype: ext4
  replication-type: none

Note that we are not limited to the ones provided. `Provider`s can be written by anyone and hosted.

> `parameters`: for defining the per-provisioner specification of volume properties.

Examples of different internal provisioners can be found [here](https://kubernetes.io/docs/concepts/storage/storage-classes/#aws-ebs).

> `reclaimPolicy`: stipulates what should be done with the created `PersistentVolume` once it is freed from `PVC`.

As mentioned previously, the option to `Delete` or `Retain` is available.

## Expandable Volumes

> The option of expanding volumes dynamically was introduced with `k8s` 1.11.

This occurs when the storage requirements in the `PVC` are changed and a new config is `apply`ied.

> The easiest way to exploit this feature is to use an internal cloud provider.

__Simply set `.allowVolumeExpansion: true` in the `StorageClass` definition.__

Below is a list of `providers` that support expandable volumes.

![](./images/expandable-volumes.png)

## Storage Summary

Here is a rough guideline for handling storage.

> __Decide if your data should be shared between containers or pods.__

If it is the former, should the data be preserved after pod termination?

__If yes, use `ephemeral volumes` to simply data exchange between applications (`LAMP` example above).__

__If the data should be shared between pods or preserved, use `PersistentVolumes`.__

Thereafter, determine if dynamic provisioning is required (it is difficult to know beforehand how much storage is required).

If not, create the following:
- `PersistentVolume` `.yml` config (defines how a `volume` is created).
- `PersistentVolumeClaim` `.yml` config (specifies how a `volume` is requested).
- `MyApplication` `.yml` config (the `workload resource`; __remember to avoid bare pods__).

If, however, dynamic provisioning is required, add another `.yml` config file to the steps above:
- `StorageClass` `.yml` config (acts as a template for giving out `PersistentVolume`s to pods in need).

> __This should be used for large-scale apps, where 'by-hand' provisioning is not feasible due to the following reasons:__

- There are many pods requesting a high storage allocation.
- It is impossible to know beforehand how many pods will run.

> Note that cloud-storage providers are preferred in this case, __as local storage may be insufficient for large deployments.__


*Other options*

One can also use the aforementioned `expandable volumes` if
- the maximum number of pods that will run at any given time is known.
- the amount of storage required for each pod is unknown.

## StatefulSets
   
Recall that `workload`s are used for __stateless applications__ (e.g. the ones not writing to external storage).

In a nutshell, StatefulSets are a type of workload resource (that manages pods) that adds a storage volume to the pod. This way, all the data associated with the pod will be stored and can be reused the next time the same pod is run, hence the name 'Stateful'. This is useful, for example, for running a pod that depends on a database where the data is stored (e.g. an SQL database).

### Limitations

- `Storage` must be provisioned by an admin (or by `StorageClass` dynamically).
- __Deleting will not delete the `Storage`__ (storage preservation > automatic purging).
- __Headless `Service`,__ which is used for the network identity of `POD`s, __must be created.__

### Components

Here are the `.yaml` definitions necessary for a `StatefulSet`:

```
apiVersion: v1
kind: Service
metadata:
  name: nginx
  labels:
    app: nginx
spec:
  ports:
  - port: 80
    name: web
  clusterIP: None
  selector:
    app: nginx
---
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: web
spec:
  serviceName: "nginx"
  replicas: 2
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: k8s.gcr.io/nginx-slim:0.8
        ports:
        - containerPort: 80
          name: web
        volumeMounts:
        - name: www
          mountPath: /usr/share/nginx/html
  volumeClaimTemplates:
  - metadata:
      name: www
    spec:
      accessModes: [ "ReadWriteOnce" ]
      resources:
        requests:
          storage: 1Gi


- A Headless Service named `nginx` is used to control the network domain.
- The `StatefulSet` named web has a Spec that indicates that three replicas of the `nginx` container will be launched in unique pods.
- The `volumeClaimTemplates` will provide stable storage using the `PersistentVolumes` provisioned by a `PersistentVolume` provisioner.

### Example

1. Create a .yaml file with the configuration above for creating a PersistentVolume resource.
2. `apply` it using the right `kubectl`.
3. Go to the dashboard, and observe the PVCs.

In [15]:
!kubectl

service/nginx created
statefulset.apps/web created


The dashboard's appearance should be similar to that shown below:
<p align=center><img src=images/PVC.png></p>

Observe that the volumes are already bound.

## Conclusion
At this point, you should have a good understanding of

- the merits of volumes over pods and containers.
- the types of volumes and their benefits.
- StorageClasses and how to handle storage.
- StatefulSets and their limitations.

## Further Reading

- Read about `DefaultStorageClass` admission plugin and how it changes behaviour for __unspecified `StorageClassses`__ in the case of `PV` or `PVC`.
- Learn about the `Volume Snapshot` feature of `PersistentVolume`s [here](https://kubernetes.io/docs/concepts/storage/volume-snapshots/).
- Read up on how to create __non-empty `PersistentVolume`s.__ Explore the information in the [volume populators](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#volume-populators-and-data-sources) section.
- Find out how to monitor the health of your `Volume`s [here](https://kubernetes.io/docs/concepts/storage/volume-health-monitoring/).